# **Step 1: Preprocess Resume Text**

In [ ]:
!pip install pdfminer.six


In [ ]:
from google.colab import files

# This will prompt a file upload dialog
uploaded = files.upload()


Saving Hasanrza Hasanli digital marketing intern.pdf to Hasanrza Hasanli digital marketing intern.pdf


In [ ]:
from pdfminer.high_level import extract_text
import io

# Get the filename (e.g., your resume)
resume_path = list(uploaded.keys())[0]

# Extract raw text from the PDF
resume_text = extract_text(resume_path)

# Show a preview
print(resume_text)


Hasanrza Hasanli 





Email: hasanrza.888@gmail.com 



Phone: (+994) 775773133 

Website: https://github.com/hasanrza888 



LinkedIn: hasanrzahasanli 

Nationality: Azerbaijani 

ABOUT ME

Digital marketing intern
Detail-oriented and analytical professional with a background in Computer Engineering and Artiﬁcial
Intelligence. Experienced in business process analysis, software quality assurance, and data-driven
decision-making. Strong knowledge of BPMN, SQL, API fundamentals, and functional testing.
Passionate about optimizing business workﬂows and implementing innovative solutions.

EDUCATION AND TRAINING

[ Sep 2023 – Current ] 

 Master’s in Artiﬁcial Intelligence 

Azerbaijan State Oil and Industry University 

City: Baku  |  Country: Azerbaijan  |  

• Focused on AI applications in business and analytics
• Researching the use of computer vision in smart systems
• Studying data modeling, machine learning, and intelligent systems

[ Sep 2019 – Jun 2023 ] 

 Bachelor’s in Compu

# **Clean the Text**

In [ ]:
import re
import string

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove emails and URLs
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers (optional)
    text = re.sub(r'\d+', '', text)
    # Collapse multiple whitespaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_resume = clean_text(resume_text)

print(cleaned_resume[:1000])  # Preview cleaned text


hasanrza hasanli   email  phone website  linkedin hasanrzahasanli nationality azerbaijani about me digital marketing intern detailoriented and analytical professional with a background in computer engineering and artiﬁcial intelligence experienced in business process analysis software quality assurance and datadriven decisionmaking strong knowledge of bpmn sql api fundamentals and functional testing passionate about optimizing business workﬂows and implementing innovative solutions education and training sep – current master’s in artiﬁcial intelligence azerbaijan state oil and industry university city baku country azerbaijan • focused on ai applications in business and analytics • researching the use of computer vision in smart systems • studying data modeling machine learning and intelligent systems sep – jun bachelor’s in computer engineering azerbaijan state oil and industry university city baku country azerbaijan • specialized in software development data analysis and system ar

# **Step 2: Use NER to Extract Resume Info**

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Load NLP Model**

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(cleaned_resume)

# Preview all named entities
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")


 (PERSON)
azerbaijan state (ORG)
baku (GPE)
• (PRODUCT)
azerbaijan state (ORG)
baku (GPE)
kanban uml diagrams (ORG)
jira trello (PERSON)
mar – mar (PERSON)
baku (GPE)
azerbaijan (GPE)
karaca (GPE)
azerbaijan city (GPE)
baku (GPE)
karaca azerbaijan (ORG)
• (PRODUCT)
oct – feb (DATE)
gurkan • (ORG)
pasha bank • (ORG)
azerbaijan (ORG)
english (NORP)


# **Step 3:Feature Vectors for Matching**

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/lang-uk/recruitment-dataset-job-descriptions-english/data/train-00000-of-00001.parquet")


In [ ]:
df.head()

,Position,Long Description,Company Name,Exp Years,Primary Keyword,English Level,Published,Long Description_lang,id,__index_level_0__
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,MyCointainer,2y,Sysadmin,intermediate,2020-10-01T00:00:00+03:00,en,c0ca96e7-85df-50df-a64e-d934cd02a170,27461
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",TechScout.tech,2y,.NET,intermediate,2022-03-01T00:00:00+02:00,en,64f4b7ea-36e4-5bdd-a8b1-185f32f7dc7f,27462
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,Innoteka,5y,JavaScript,fluent,2021-07-01T00:00:00+03:00,en,b9a1303e-dd0c-5ed1-8f62-be2bc4c7da4f,27463
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",FirstFive,2y,Marketing,upper,2022-01-01T00:00:00+02:00,en,99cb3f4a-9b4b-53d9-9a3b-bab2c22da346,27464
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",MimirB2B,1y,Marketing,upper,2021-12-01T00:00:00+02:00,en,bc1419f7-28e2-582b-8d53-22e28b2f0210,27465


In [ ]:
job_titles = df["Position"].fillna("").tolist()
job_descriptions = df["Long Description"].fillna("").tolist()


**cleaning**

In [ ]:
def clean_job_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Collapse whitespace
    return text.strip()

job_descriptions_cleaned = [clean_job_text(desc) for desc in job_descriptions]3

In [ ]:
job_descriptions_cleaned[:1]

['requirements we re looking for a long term collaboration with someone that has an experience in crypto masternodes nodes validators etc we need to set up kyber network nebulas secretnetwork tron aion defichain eos tomochain elrond irisnet iotex terra chainx thorchain succesful candidates will have an opportunity to get more jobs and long term collaboration']

**Combine Resume with Job Descriptions**

In [ ]:
documents = [cleaned_resume] + job_descriptions_cleaned


In [ ]:
documents[:1]

['hasanrza hasanli \uf0e0 \uf0ac email \uf879 phone website \uf08c linkedin hasanrzahasanli nationality azerbaijani about me digital marketing intern detailoriented and analytical professional with a background in computer engineering and artiﬁcial intelligence experienced in business process analysis software quality assurance and datadriven decisionmaking strong knowledge of bpmn sql api fundamentals and functional testing passionate about optimizing business workﬂows and implementing innovative solutions education and training sep – current master’s in artiﬁcial intelligence azerbaijan state oil and industry university city baku country azerbaijan • focused on ai applications in business and analytics • researching the use of computer vision in smart systems • studying data modeling machine learning and intelligent systems sep – jun bachelor’s in computer engineering azerbaijan state oil and industry university city baku country azerbaijan • specialized in software development data 

**TF-IDF Vectorization and Similarity**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorize
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Compute similarity between resume and all job descriptions
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()


**Top 5 Job Recommendations**

In [ ]:
import numpy as np

top_indices = cosine_similarities.argsort()[-5:][::-1]  # Top 5 descending
top_matches = [(job_titles[i], cosine_similarities[i]) for i in top_indices]

print("Top 5 Job Recommendations:")
for title, score in top_matches:
    print(f"{title} - Score: {score:.4f}")


Top 5 Job Recommendations:
Head of MarTech and Analytics - Score: 0.2486
Business System Analyst, Investment Management Solutions - Score: 0.2450
Business System Analyst, Investment Management Solutions - Score: 0.2398
Lead/Expert Data Analyst - Score: 0.2360
Country Manager - Score: 0.2358
